In [10]:
%matplotlib qt
import os
import pickle
from sdypy import EMA
import numpy as np
import matplotlib.pyplot as plt 
from EMA_functions import *
import pandas as pd
import matplotlib.animation as animation
import ast
from matplotlib.patches import Ellipse
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import matplotlib.colors as col

In [21]:
df = pd.read_csv('H:/My Drive/PHD/HSC/file_descriptions_wlocs.csv')
# file_name_vec = ['Full_web_ecc0_new_Floc9_v0', 
#                  'Full_web_ecc0_new_Floc9_v1', 
#                  'Full_web_ecc1_new_Floc1_v4',
#                  'Full_web_ecc1_new_Floc1_v5',
#                  'Full_web_ecc2_new_Floc7_v0',
#                  'Full_web_ecc2_new_Floc7_v1']
v = 1
if v == 0:
    file_name_vec = ['Full_web_ecc0_new_Floc9_v1', 
                    'Full_web_ecc1_new_Floc1_v5',
                    'Full_web_ecc2_new_Floc7_v0']
elif v == 1:
    file_name_vec = ['Full_web_ecc0_new_Floc9_v0',
                    'Full_web_ecc0_new_Floc9_v1',
                    'Full_web_ecc1_new_Floc1_v4',
                    'Full_web_ecc1_new_Floc1_v5',
                    'Full_web_ecc2_new_Floc7_v0',
                    'Full_web_ecc2_new_Floc7_v1']
    
file_name_vec = ['Full_web_ecc0_new_Floc9_v0',
                'Full_web_ecc0_new_Floc9_v1',
                'Full_web_ecc2_new_Floc7_v0',
                'Full_web_ecc2_new_Floc7_v1']
file_name_vec_video = [x + '_S01.cihx' for x in file_name_vec]
df_filtered = df[df['filename'].isin(file_name_vec_video)]

roi_size = (9, 9)
reference_image = (0, 30)
df_filtered

,filename,path,ecc,loc,mass,taut/loose,impact/pluck,version,n_frames,fps,Oversatured pixels,Undersatured pixels,prey_ij,spider_ij
92,Full_web_ecc2_new_Floc7_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web0\rev1\Full_web_ec...,2,7,True,Loose,Impact,0,7000,8000.0,493,19299,"(693, 328)","(505, 211)"
93,Full_web_ecc2_new_Floc7_v1_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web0\rev1\Full_web_ec...,2,7,True,Loose,Impact,1,7000,8000.0,422,19328,"(689, 330)","(505, 211)"
195,Full_web_ecc0_new_Floc9_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web0\Full_web_ecc0_ne...,0,9,True,Loose,Impact,0,10000,8000.0,5430,20745,"(664, 330)","(529, 268)"
196,Full_web_ecc0_new_Floc9_v1_S01.cihx,H:\My Drive\PHD\HSC\ecc0\web0\Full_web_ecc0_ne...,0,9,True,Loose,Impact,1,10000,8000.0,4604,21139,"(661, 333)","(529, 268)"


In [23]:
# spider_ij_new = [(534, 277), (525, 206), (507, 223)]
# spider_ij_new = [(534, 277), (534, 277), (525, 206), (525, 206), (507, 223), (507, 223)]
spider_ij_new = [(534, 277),(534, 277),(507, 223), (507, 223)]
# 
n_modes = 8
fn = np.zeros((len(file_name_vec), n_modes))
xi = np.zeros((len(file_name_vec), n_modes))
color_vec = ['r', 'r', 'b', 'b', 'k', 'k']
# color_vec = ['r', 'b', 'k']
ls_vec = ['-', ':', '-', '-', '-', '-']
fig, ax = plt.subplots(2, 1)
ax[0].set_xlim([0, 25])
ax[1].set_xlim([0, 25])
for i, (file_name, file_name_video) in enumerate(zip(file_name_vec, file_name_vec_video)):
    row = df_filtered[df_filtered['filename'] == file_name_video]
    row_index = row.index
    df_filtered.loc[row_index, 'spider_ij'] = str(spider_ij_new[i])
    ecc = row['ecc'].values[0]
    EMA_structure = EMA_Structure.load(file_name)
    video = EMA_structure.open_video()
    # EMA_structure.open_impact_data()
    # force_signal = EMA_structure.force
    # force_signal = np.append(force_signal, np.zeros(len(force_signal)))
    # n_f = len(force_signal)
    # fs_force = 1/(EMA_structure.t_force[1] - EMA_structure.t_force[0])
    # freq_force = np.fft.rfftfreq(n_f, 1/fs_force)
    # df_force = freq_force[1] - freq_force[0]
    # Force = np.fft.rfft(force_signal, n=n_f) / n_f
    # EMA_structure.open_displacements(roi_size=roi_size, reference_image=reference_image, auto_nut_idx=True)
    EMA_structure.open_cam_EMA()
    fn[i, :] = EMA_structure.cam.nat_freq[:n_modes]
    xi[i, :] = EMA_structure.cam.nat_xi[:n_modes]
    fig, ax = plot_FRF(EMA_structure.cam, fig, ax, c=color_vec[i], ls=ls_vec[i], annotate=True, label=f"{EMA_structure.cam.F_peak:.2f}N - Ecc {ecc}")
    print(file_name)
    print(EMA_structure.cam.F_peak)
    # fn_vec = np.arange(0, 40, df_force)
    # Force  = Force[:len(fn_vec)]
    # omega_vec = 2*np.pi*fn_vec
    # H = np.zeros_like(fn_vec, dtype=complex)
    # for f_n, xi_n in zip(EMA_structure.cam.nat_freq, EMA_structure.cam.nat_xi):
    #     omega_n = 2*np.pi*f_n
    #     # H += omega_n**2 / (omega_vec**4 - omega_vec**2*omega_n**2 - 2j*omega_n*omega_vec**3*xi_n)
    #     H += (omega_n**2 / (-omega_vec**2 + omega_n**2 + 2j*omega_n*omega_vec*xi_n)) * -1/omega_vec**2
    # Acc = H * Force
    # Acc[0]  = 0.0001
    # ax[1].semilogy(fn_vec, np.abs(Acc), c='k', ls=ls_vec[i])
    # plt.show()





H:/My Drive/PHD/HSC\ecc0\web0\Full_web_ecc0_new_Floc9_v0_S01\Full_web_ecc0_new_Floc9_v0_S01.cihx
Full_web_ecc0_new_Floc9_v0
0.8838292582716049


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 10000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\EMA.py:888: RuntimeWarning: divide by zero encountered in divide
  p1L = np.kron(np.array([1, 0]), -1/omega**2)
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\shape_base.py:1179: RuntimeWarning: invalid value encountered in multiply
  result = _nx.multiply(a_arr, b_arr, subok=(not is_any_mat))
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\EMA.py:889: RuntimeWarning: divide by zero encountered in divide
  p2L = np.kron(np.array([0, 1]), -1/omega**2)


H:/My Drive/PHD/HSC\ecc0\web0\Full_web_ecc0_new_Floc9_v1_S01\Full_web_ecc0_new_Floc9_v1_S01.cihx
Full_web_ecc0_new_Floc9_v1
1.7108269530864197


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 10000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\EMA.py:888: RuntimeWarning: divide by zero encountered in divide
  p1L = np.kron(np.array([1, 0]), -1/omega**2)
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\shape_base.py:1179: RuntimeWarning: invalid value encountered in multiply
  result = _nx.multiply(a_arr, b_arr, subok=(not is_any_mat))
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\EMA.py:889: RuntimeWarning: divide by zero encountered in divide
  p2L = np.kron(np.array([0, 1]), -1/omega**2)


H:/My Drive/PHD/HSC\ecc2\web0\rev1\Full_web_ecc2_new_Floc7_v0_S01\Full_web_ecc2_new_Floc7_v0_S01.cihx
Full_web_ecc2_new_Floc7_v0
1.2162814271604938


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 7000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\EMA.py:888: RuntimeWarning: divide by zero encountered in divide
  p1L = np.kron(np.array([1, 0]), -1/omega**2)
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\shape_base.py:1179: RuntimeWarning: invalid value encountered in multiply
  result = _nx.multiply(a_arr, b_arr, subok=(not is_any_mat))
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\EMA.py:889: RuntimeWarning: divide by zero encountered in divide
  p2L = np.kron(np.array([0, 1]), -1/omega**2)


H:/My Drive/PHD/HSC\ecc2\web0\rev1\Full_web_ecc2_new_Floc7_v1_S01\Full_web_ecc2_new_Floc7_v1_S01.cihx
Full_web_ecc2_new_Floc7_v1
1.6973610449382714


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 7000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\EMA.py:888: RuntimeWarning: divide by zero encountered in divide
  p1L = np.kron(np.array([1, 0]), -1/omega**2)
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\shape_base.py:1179: RuntimeWarning: invalid value encountered in multiply
  result = _nx.multiply(a_arr, b_arr, subok=(not is_any_mat))
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\EMA.py:889: RuntimeWarning: divide by zero encountered in divide
  p2L = np.kron(np.array([0, 1]), -1/omega**2)


In [4]:
fig, ax = plt.subplots()
for i in range(len(file_name_vec)):
    ax.plot(range(n_modes), fn[i]/fn[i,0], '*', color=color_vec[i], label=file_name_vec[i])
ax.set_xlabel('Mode')
ax.set_ylabel('Normalized natural Frequency ')
ax.legend()
ax.set_ylim([0, 7])
plt.show()


In [5]:
fig, ax = plt.subplots(3, n_modes, sharex=True, sharey=True, subplot_kw={'projection': '3d'})
fig.tight_layout()

cmap = plt.cm.get_cmap('seismic')
for i, (file_name, file_name_video) in enumerate(zip(file_name_vec, file_name_vec_video)):
    row = df_filtered[df_filtered['filename'] == file_name_video]
    ecc = row['ecc'].values[0]
    EMA_structure = EMA_Structure.load(file_name)
    # video = EMA_structure.open_video()
    # EMA_structure.open_displacements(roi_size=roi_size, reference_image=reference_image, auto_nut_idx=True)
    EMA_structure.open_cam_EMA()
    plot_MAC(EMA_structure.cam, 6)
    idx = np.where(EMA_structure.tp[EMA_structure.valid_tps] == EMA_structure.nearest_nut_index)[0]
    for j in range(n_modes):
        Z = plot_mode_shape(EMA_structure.cam, j, EMA_structure.tp, 0, find_Z=True)
        ax[ecc, j].scatter(EMA_structure.tp[EMA_structure.valid_tps][:, 1], -EMA_structure.tp[EMA_structure.valid_tps][:, 0], Z, c=Z, cmap=cmap, s=5, alpha=0.6)
        if ecc == 0:
            ax[ecc, j].set_title('Mode ' + str(j+1))
        if j == 0:
            ax[ecc, j].set_ylabel('Ecc '+ str(ecc))
            
        # ax[ecc, j].set_title('Ecc '+ str(ecc) + ' & Mode ' + str(j+1))

        


C:\Users\thijsmas\AppData\Local\Temp\ipykernel_19504\2123340332.py:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('seismic')
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\tools.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  MAC[i, j] = MAC[i, j]/\
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\tools.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  MAC[i, j] = MAC[i, j]/\
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\tools.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  MAC[i, j] = MAC[i, j]/\


In [6]:
n_modes = 5
fig, ax = plt.subplots(3, n_modes, sharex=True, sharey=True, figsize=(18, 8))
fig.tight_layout()
cmap = plt.cm.get_cmap('seismic')
for i, (file_name, file_name_video) in enumerate(zip(file_name_vec, file_name_vec_video)):
    row = df_filtered[df_filtered['filename'] == file_name_video]
    ecc = row['ecc'].values[0]
    EMA_structure = EMA_Structure.load(file_name)
    # video = EMA_structure.open_video()
    # EMA_structure.open_displacements(roi_size=roi_size, reference_image=reference_image, auto_nut_idx=True)
    EMA_structure.open_cam_EMA()
    plot_MAC(EMA_structure.cam, 6)
    idx = np.where(EMA_structure.tp[EMA_structure.valid_tps] == EMA_structure.nearest_nut_index)[0]
    for j in range(n_modes):
        Z = plot_mode_shape(EMA_structure.cam, j, EMA_structure.tp, 0, find_Z=True)
        ax[ecc, j].scatter(EMA_structure.tp[EMA_structure.valid_tps][:, 1], -EMA_structure.tp[EMA_structure.valid_tps][:, 0], c=Z, cmap=cmap, s=5, alpha=0.6)
        ax[ecc, j].set_aspect('equal')
        if ecc == 0:
            ax[ecc, j].set_title('Mode ' + str(j+1))
        if j == 0:
            ax[ecc, j].set_ylabel('Ecc '+ str(ecc))
        ax[ecc, j].set_xlabel(str(np.round(EMA_structure.cam.nat_freq[j], 2)) + ' Hz')
        ax[ecc, j].set_xticks([])
        ax[ecc, j].set_yticks([])

C:\Users\thijsmas\AppData\Local\Temp\ipykernel_19504\3687992988.py:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('seismic')
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\tools.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  MAC[i, j] = MAC[i, j]/\
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\tools.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  MAC[i, j] = MAC[i, j]/\
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\tools.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  MAC[i, j] = MAC[i, j]/\


In [7]:
n_modes = 6
fig, ax = plt.subplots(3, n_modes, figsize=(18, 8))
fig2 = plt.figure(figsize=(18, 10))
gs = gridspec.GridSpec(3, 2, width_ratios=[4, 1])
ax2 = [[], [], []]
for i in range(3):
    for j in range(2):
        ax2[i].append(plt.subplot(gs[i, j]))
fig.tight_layout()
plt.show()

fig.tight_layout()
cmap = plt.cm.get_cmap('seismic')
width_pixels = 250
height_pixels = 150
width_elipse = 130
height_elipse = 60

n_sections = 8

cmap_circ = cm.get_cmap('Accent')
norm = col.Normalize(vmin=0, vmax=1)
sm = cm.ScalarMappable(norm = norm, cmap=cmap_circ)

for i, (file_name, file_name_video) in enumerate(zip(file_name_vec, file_name_vec_video)):
    row = df_filtered[df_filtered['filename'] == file_name_video]
    ecc = row['ecc'].values[0]
    EMA_structure = EMA_Structure.load(file_name)
    # video = EMA_structure.open_video()
    # EMA_structure.open_displacements(roi_size=roi_size, reference_image=reference_image, auto_nut_idx=True)
    EMA_structure.open_cam_EMA()
    idx = np.where(EMA_structure.tp[EMA_structure.valid_tps] == EMA_structure.nearest_nut_index)[0]
    for j in range(n_modes):
        Z = plot_mode_shape(EMA_structure.cam, j, EMA_structure.tp, 0, find_Z=True)
        ax[ecc, j].scatter(EMA_structure.tp[EMA_structure.valid_tps][:, 1], -EMA_structure.tp[EMA_structure.valid_tps][:, 0], c=Z, cmap=cmap, s=5, alpha=0.6)
        if ecc == 0:
            ax[ecc, j].set_title('Mode ' + str(j+1))
        if j == 0:
            ax[ecc, j].set_ylabel('Ecc '+ str(ecc) + f" - {EMA_structure.cam.F_peak:.2f}N")
        ax[ecc, j].set_xlabel(str(np.round(EMA_structure.cam.nat_freq[j], 2)) + ' Hz')
        ax[ecc, j].set_xticks([])
        ax[ecc, j].set_yticks([])
        spider_ij = ast.literal_eval(row['spider_ij'].values[0])
        ax[ecc, j].set_xlim([spider_ij[0]-width_pixels/2, spider_ij[0]+width_pixels/2])
        ax[ecc, j].set_ylim([-spider_ij[1]-height_pixels/2, -spider_ij[1]+height_pixels/2])
        prey_ij = ast.literal_eval(row['prey_ij'].values[0])
        quiver_scale = 1 / np.linalg.norm([prey_ij[0]-spider_ij[0], prey_ij[1]-spider_ij[1]])
        ax[ecc, j].quiver(spider_ij[0], -spider_ij[1], (prey_ij[0]-spider_ij[0])*quiver_scale, -(prey_ij[1]-spider_ij[1])*quiver_scale, angles='xy', scale_units='xy', scale=0.01, color='black')
        # ax[ecc, j].add_patch(Ellipse((spider_ij[0], -spider_ij[1]), width_elipse, height_elipse, edgecolor='red', facecolor='none'))
        
    ellipse_center = (spider_ij[0], -spider_ij[1])
    a = width_elipse / 2
    b = height_elipse / 2

    inside_ellipse = (EMA_structure.tp[:,1] - ellipse_center[0])**2/a**2 + (-EMA_structure.tp[:,0] - ellipse_center[1])**2/b**2 <=1
    inside_and_valid = np.logical_and(inside_ellipse, EMA_structure.valid_tps)
    points_inside_ellipse = EMA_structure.tp[inside_and_valid]
    angles = np.arctan2(-(points_inside_ellipse[:, 0] - spider_ij[1]), points_inside_ellipse[:, 1] - spider_ij[0])
    # ax[ecc, j].scatter(points_inside_ellipse[:, 1], -points_inside_ellipse[:, 0], c=angles, cmap=cmap_circ, s=5)

    frf_all = EMA_structure.cam.frf[inside_ellipse[EMA_structure.valid_tps]]
    
    angle_prey = np.arctan2(-(prey_ij[1] - spider_ij[1]), prey_ij[0] - spider_ij[0])
    if angle_prey < 0:
        angle_prey += 2*np.pi
    angle_prey = angle_prey/(2*np.pi)
    angles[angles < 0] = angles[angles < 0] + 2*np.pi
    normalized_angles = angles/(2*np.pi)
    bucket_edges = np.linspace(-1/(2*n_sections), 1-1/(2*n_sections), num=n_sections + 1) + angle_prey
    bucket_edges = bucket_edges - np.floor(bucket_edges)
    bucket_edges[bucket_edges > 1] -= 1
    bucket_edges[0] = 0
    bucket_edges = np.append(bucket_edges,1)
    bucket_edges = np.sort(bucket_edges)
    angle_sections = (np.digitize(normalized_angles, bucket_edges, right = True)-1).astype(int)
    angle_sections[angle_sections == n_sections] = 0
        
    for freq in EMA_structure.cam.nat_freq[:n_modes]:
        ax2[ecc][0].axvline(x=freq, color='k')
    ax2[ecc][0].set_ylabel(f'$|H|$ - Ecc {ecc}')
    if ecc == 2:
        ax2[ecc][0].set_xlabel('Frequency [Hz]')

    for section in range(8):
        current_points = angle_sections == section
        frf_section = frf_all[current_points]
        if len(frf_section) <= 5:
            lt = ':'
        else:
            lt = '-'
        ax2[ecc][0].semilogy(EMA_structure.cam.freq, np.abs(np.average(frf_section,0)), color=sm.to_rgba(section/(n_sections-1)), linestyle=lt)
        ax2[ecc][1].scatter(points_inside_ellipse[current_points, 1], -points_inside_ellipse[current_points, 0], c=sm.to_rgba(section/(n_sections-1)), s=5)
    ax2[ecc][1].quiver(spider_ij[0], -spider_ij[1], (prey_ij[0]-spider_ij[0])*quiver_scale, -(prey_ij[1]-spider_ij[1])*quiver_scale, angles='xy', scale_units='xy', scale=0.01, color='black')
    ax2[ecc][1].set_xticks([])
    ax2[ecc][1].set_yticks([])
    ax2[ecc][0].set_xlim([0, 35])
    ax2[ecc][1].set_aspect('equal')
    for edge in bucket_edges[1:-1]:
        x_values = [spider_ij[0], spider_ij[0] + 70*np.cos(edge*2*np.pi)]
        y_values = [-spider_ij[1], -spider_ij[1] + 70*np.sin(edge*2*np.pi)]
        ax2[ecc][1].plot(x_values, y_values, color='black', alpha=0.5, linewidth=0.5)
    # ax2[ecc][1].scatter(spider_ij[0] + 10*np.cos(bucket_edges*2*np.pi), -spider_ij[1] + 10*np.sin(bucket_edges*2*np.pi), c='black', s=10)

    # for frf_i, frf in enumerate(frf_all):
    #     ax2[ecc][0].semilogy(EMA_structure.cam.freq, np.abs(frf), color=colors[frf_i], linewidth=0.3)
    #     ax2[ecc][0].set_ylabel(f'$|H|$ - Ecc {ecc}')
    #     if ecc == 2:
    #         ax2[ecc][0].set_xlabel('Frequency [Hz]')
    #     ax2[ecc][0].set_xlim([12, 20])
    #     ax2[ecc][1].scatter(points_inside_ellipse[:, 1], -points_inside_ellipse[:, 0], c=normalized_angles, cmap=cmap_circ, s=5)
    #     ax2[ecc][1].quiver(spider_ij[0], -spider_ij[1], (prey_ij[0]-spider_ij[0])*quiver_scale, -(prey_ij[1]-spider_ij[1])*quiver_scale, angles='xy', scale_units='xy', scale=0.01, color='black')
    #     ax2[ecc][1].set_xticks([])
    #     ax2[ecc][1].set_yticks([])

# Save the figures
fig.savefig(os.path.join('G:/.shortcut-targets-by-id/1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx/Illimited Lab Projects/Research Projects/Spiders/Simulations', 'comparison', 'prey_angle_' + str(int(angle_prey*360)) + f'deg_local_modes_v{v}.png'))
fig2.savefig(os.path.join('G:/.shortcut-targets-by-id/1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx/Illimited Lab Projects/Research Projects/Spiders/Simulations', 'comparison', 'prey_angle_' + str(int(angle_prey*360)) + f'deg_FRFs_v{v}.png'))

for i in range(3):
    for j in range(6):
        ax[i, j].set_xlim([0, 1024])
        ax[i, j].set_ylim([-512, 0])
fig.savefig(os.path.join('G:/.shortcut-targets-by-id/1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx/Illimited Lab Projects/Research Projects/Spiders/Simulations', 'comparison', 'prey_angle_' + str(int(angle_prey*360)) + f'deg_global_modes_v{v}.png'))




C:\Users\thijsmas\AppData\Local\Temp\ipykernel_19504\3144133231.py:12: UserWarning: The figure layout has changed to tight
  fig.tight_layout()
C:\Users\thijsmas\AppData\Local\Temp\ipykernel_19504\3144133231.py:13: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('seismic')
C:\Users\thijsmas\AppData\Local\Temp\ipykernel_19504\3144133231.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap_circ = cm.get_cmap('Accent')
C:\Users\thijsmas\AppData\Local\Temp\ipykernel_19504\3144133231.py:92: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence

In [8]:
n_modes = 6
fig, ax = plt.subplots(figsize=(18, 8))
fig.tight_layout()
cmap = plt.cm.get_cmap('seismic')

for i, (file_name, file_name_video) in enumerate(zip(file_name_vec, file_name_vec_video)):
    row = df_filtered[df_filtered['filename'] == file_name_video]
    ecc = row['ecc'].values[0]
    EMA_structure = EMA_Structure.load(file_name)
    EMA_structure.open_cam_EMA()


    
    H = EMA_structure.cam.H[inside_and_valid]


C:\Users\thijsmas\AppData\Local\Temp\ipykernel_19504\2190275541.py:4: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('seismic')


IndexError: boolean index did not match indexed array along dimension 0; dimension is 1686 but corresponding boolean dimension is 2034